In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

In [ ]:
WORKING_DIRECTORY = './drive/MyDrive/Colab\ Notebooks/diploma/'

In [12]:
 # Создаём SparkSession
spark = SparkSession.builder \
    .appName("PetCo") \
    .getOrCreate()

In [10]:
# Read dataset with beh logs
data_items = spark.read.parquet(WORKING_DIRECTORY+'data/data_set_items')

In [11]:
data_items.show()

+----------+---------------+--------------------+-----+-----------+--------------------+--------------------+--------------+-------------+--------------------+------------+-------------------+-------------------+----------+--------------------+
|        id|autocomplete_id|                name|score|customer_id|          name_lower|       metadata_json|computed_score|boost_or_bury|    ds_metadata_json|section_name|         created_at|         updated_at|       day|              ac_key|
+----------+---------------+--------------------+-----+-----------+--------------------+--------------------+--------------+-------------+--------------------+------------+-------------------+-------------------+----------+--------------------+
|3668992957|           1947|Kaytee Forti-Diet...|   -1|    5017277|kaytee forti-diet...|{"url": "/shop/en...|         68803|            0|{"weighted_keywor...|    Products|2022-05-16 14:44:33|2024-06-26 02:09:22|2024-06-26|key_afiSr5Y4gCaaSW5X|
|3668994348|        

In [ ]:
data_items.printSchema()  # Структура данных

root
 |-- id: long (nullable = true)
 |-- autocomplete_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- name_lower: string (nullable = true)
 |-- metadata_json: string (nullable = true)
 |-- computed_score: integer (nullable = true)
 |-- boost_or_bury: integer (nullable = true)
 |-- ds_metadata_json: string (nullable = true)
 |-- section_name: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- day: string (nullable = true)
 |-- ac_key: string (nullable = true)



In [ ]:
print(f'Число строк: {data_items.count()}')

Число строк: 4900493


In [ ]:
n_items = data_items.select('customer_id').distinct().count()
print(f'Число товаров в каталоге: {n_items}')

Число товаров в каталоге: 32001


In [ ]:
one_item = data_items.filter(data_items.customer_id == 9082)
one_item.show()
print(one_item.count())

+----------+---------------+--------------------+-----+-----------+--------------------+--------------------+--------------+-------------+--------------------+------------+-------------------+-------------------+----------+--------------------+
|        id|autocomplete_id|                name|score|customer_id|          name_lower|       metadata_json|computed_score|boost_or_bury|    ds_metadata_json|section_name|         created_at|         updated_at|       day|              ac_key|
+----------+---------------+--------------------+-----+-----------+--------------------+--------------------+--------------+-------------+--------------------+------------+-------------------+-------------------+----------+--------------------+
|      NULL|           1947|Wellness Old Moth...|   -1|       9082|wellness old moth...|{"url": "/shop/en...|          9373|         NULL|{"weighted_keywor...|    Products|2022-05-16 14:44:33|2024-11-29 01:06:06|2024-11-29|key_afiSr5Y4gCaaSW5X|
|      NULL|        

In [ ]:
one_item.select('metadata_json').collect()[0].metadata_json

'{"url": "/shop/en/petcostore/product/everyyay-bowl-li-for-dogs", "image_url": "https://assets.petco.com/petco/image/upload/f_auto,q_auto,w_190/dpr_auto/3764305-center-2", "facets": {"How to get it": ["Same Day Delivery", "Free Pickup Today", "One Time Delivery"], "Primary Brand": ["EveryYay"], "Primary Color": ["Gray"], "Killed Item flag": ["Yes"], "Primary Pet Type": ["Cat,Dog"], "Personalized Item flag": ["No"], "Prescription Medication": ["No"]}, "group_ids": ["buy-online-pick-up-in-store-dog-products", "cat-feeding-scoops", "same-day-delivery-cat-products", "dog-food-containers", "dog-bowls", "buy-online-pick-up-in-store-cat-products", "same-day-delivery-dog-products", "dog-feeding-supplies-deals"], "PTC_OMNI_REPEAT_DELIVERY_FL": "No", "parentCatEntryIDAsString": "6697520", "PTC_OMNI_PRIMARY_ITEM_FLAG": "Yes", "mfName": "EveryYay", "PTC_OMNI_IN_STORE_ONLY_FLAG": "No", "PTC_OMNI_PROP_65_FLAG": "No", "PTC_OMNI_TAXONOMY": "Storage|Containers|Non-Electronic Containers", "PTC_OMNI_PERS

In [ ]:
one_item.select('metadata_json').collect()[-1]

Row(metadata_json='{"url": "/shop/en/petcostore/product/everyyay-bowl-li-for-dogs", "image_url": "https://assets.petco.com/petco/image/upload/f_auto,q_auto,w_190/dpr_auto/3764305-center-2", "facets": {"How to get it": ["Same Day Delivery", "Free Pickup Today", "One Time Delivery"], "Primary Brand": ["EveryYay"], "Primary Color": ["Gray"], "Killed Item flag": ["Yes"], "Primary Pet Type": ["Cat,Dog"], "Personalized Item flag": ["No"], "Prescription Medication": ["No"]}, "group_ids": ["dog-bowls", "dog-feeding-supplies-deals", "gifts-under-10", "gifts-under-20", "buy-online-pick-up-in-store-dog-products", "everyday-low-price-beds-bowls", "dog-food-containers", "same-day-delivery-cat-products", "cat-feeding-scoops", "same-day-delivery-dog-products", "buy-online-pick-up-in-store-cat-products"], "UPC_NUMBER": "800443651568", "PTC_OMNI_PRIMARY_ITEM_FLAG": "Yes", "PTC_OMNI_REPEAT_DELIVERY_FL": "No", "mfName": "EveryYay", "deactivated": false, "itemimg": "https://assets.petco.com/petco/image/up

In [ ]:
# Группируем по 'customer_user_id' и считаем количество строк для каждого 'customer_user_id'
user_counts_purchased = data_purchases.groupBy("customer_user_id").count()
average_count_purchases = user_counts_purchased.agg(F.avg("count")).collect()[0][0]
median_count_purchases = user_counts_purchased.approxQuantile('count', [0.5], 0.01)[0]

print(f"Среднее число покупок: {round(average_count_purchases, 1)}")
print(f'Медианное число покупок: {median_count_purchases}')

# **Вывод:**

В каталоге 32 001 товар (разные customer_id). Еще есть разные вариации товара внутри одного customer_id (в среднем около 150), информация об этих вариациях содержится (наверное) в metadata.